In [1]:
import mdtraj as md

In [2]:
from pathlib import Path

In [3]:
!pwd

/home/crystal/Desktop/L-ile


In [4]:
traj = md.load_dcd('L_prd.dcd', top='L_clean.pdb')

In [5]:
traj

<mdtraj.Trajectory with 2000000 frames, 41 atoms, 2 residues, and unitcells at 0x7f12852f33a0>

In [6]:
traj = md.Trajectory.superpose(traj, traj[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

In [7]:
psi_indices, phi_indices = [17,  1,  0, 21], [ 0, 21, 24, 23]
angles = md.compute_dihedrals(traj, [phi_indices, psi_indices])
print(angles)

[[-2.1284437  2.0694518]
 [-2.1462746  1.9710411]
 [-2.0160308  1.906835 ]
 ...
 [-1.5935591 -1.1644721]
 [-1.0255878 -1.3852625]
 [-1.3776194 -1.2261732]]


In [8]:
from pylab import *
from math import pi
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

psi = angles[:, 1].flatten()
phi = angles[:, 0].flatten()

fig, axes = plt.subplots(2,1, figsize=(20,8))

axes[0].plot(psi, lw=0.5)
axes[1].plot(phi, lw=0.5)
axes[0].set_title('L-Isoleucine dipeptide', fontsize =18)
axes[0].set_ylabel(r'$\Psi$ Angle [radians]')
axes[1].set_ylabel(r'$\Phi$ Angle [radians]')
axes[1].set_xlabel('Time (ns)')
axes[0].set_xlabel('Time (ns)')

for ax in axes:
    ax.set_xticks(ticks=np.arange(0, int(angles.shape[0])+1, 10000), labels=np.arange(0, int(angles.shape[0]/100)+1, 100))
    ax.set_xlim([0, angles.shape[0]])
plt.savefig('L-Isoleucine dipeptide_transition.png', dpi=600)

In [9]:
psi = angles[:, 1].flatten()
phi = angles[:, 0].flatten()

import pyemma as pm
from pylab import *
from math import pi
fig, ax = plt.subplots(figsize=(7,6),tight_layout=True)
fes = pm.plots.plot_free_energy(angles[:, 0], angles[:, 1], nbins=100, ax=ax)
ax.set_xlabel(r'$\Phi$ Angle (degree)', fontsize =15)
ax.set_ylabel(r'$\Psi$ Angle (degree)', fontsize =15)
ax.set_title('L-Isoleucine dipeptide', fontsize =18)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.show()

phi < -1

phi < -1
psi < 0.5 ; psi > -1

s1_mask = np.logical_and(np.logical_or(psi>-1.5, psi< -0.5), phi<-1)

s2_mask = np.logical_and(np.logical_or(psi<-3, psi> 1), phi<-1)

In [10]:
def find_transitions(s1_mask, s2_mask):
    s2s1_transition_ids, s1s2_transition_ids = [], []
    for i in tqdm(range(1, len(s1_mask)), desc=f'Finding transitions'):
        s1_ids = [k for k, x in enumerate(s1_mask[:i]) if x]
        s2_ids = [k for k, x in enumerate(s2_mask[:i]) if x]
        if len(s1_ids) == 0 or len(s2_ids) == 0:
            continue
        if s1_mask[i] and (s1_ids[-1] < s2_ids[-1]):
            s2s1_transition_ids.append([s2_ids[-1], i])
            with open('1sts2s1.txt', 'a') as f:
                # Write the list of strings to the file
                f.writelines(str(s2_ids[-1]) + ' ' + str(i) + '\n')
        if s2_mask[i] and (s1_ids[-1] > s2_ids[-1]):
            s1s2_transition_ids.append([s1_ids[-1], i])
            with open('1sts1s2.txt', 'a') as f:
                # Write the list of strings to the file
                f.writelines(str(s1_ids[-1]) + ' ' + str(i) + '\n')
    return s2s1_transition_ids, s1s2_transition_ids

In [11]:
def save_transitions(traj, s2s1_transition_ids, s1s2_transition_ids):
    if len(s2s1_transition_ids)>0:
        for no, ids in enumerate(s2s1_transition_ids):
            transition_traj = traj[ids[0]:ids[1]+1]
            transition_traj.superpose(transition_traj[0])
            transition_traj.save_pdb(md_transition_path.joinpath(f'traj_s2s1slowest{no}.pdb'))
    if len(s1s2_transition_ids)>0:
        for no, ids in enumerate(s1s2_transition_ids):
            transition_traj = traj[ids[0]:ids[1]+1]
            transition_traj.superpose(transition_traj[0])
            transition_traj.save_pdb(md_transition_path.joinpath(f'traj_s1s2slowest{no}.pdb'))
    return None

In [12]:
from tqdm import tqdm

In [13]:
md_transition_path = Path('./')

find_transitions(phi < -1, phi > 1)

In [14]:
s2s1_transition_ids = [[56191, 56196],
[69680, 69687],
[192298, 192299],
[331657, 331668],
[558618, 558619],
[642901, 642902],
[643079, 643080],
[724614, 724615],
[943656, 943689],
[959291, 959300],
[960872, 960886],
[1121946, 1121972],
[1219127, 1219128],
[1427650, 1427651],]
s1s2_transition_ids =  [[65623, 65631],
[192297, 192298],
[330693, 330696],
[558617, 558618],
[642900, 642901],
[643078, 643079],
[724613, 724614],
[937931, 937933],
[959040, 959046],
[960135, 960143],
[1120794, 1120801],
[1219126, 1219127],
[1427649, 1427650],]
save_transitions(traj, s2s1_transition_ids, s1s2_transition_ids)

traj0 = md.load_pdb('traj_s1s2slowest1.pdb')

md.compute_phi(traj0)